In [23]:
from scipy.stats import qmc
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import shutil
import os
from sklearn.metrics import mean_squared_error
from math import sqrt
import subprocess
import scienceplots

import warnings
import matplotlib.pyplot as plt
import random

warnings.filterwarnings("ignore")

## Calibration strategy:
- Calibrate on both june and july
- Leave out 10-20% 
- Each sensor within each week is its own dataset
- Leave out a random sensor from calibration
- Consider all sensors as data set (22 sensors) remove 20%, then run calibration to minimize rmse, Then run validation on the sensors that were left out for both week in june and july
- Leave latin hypercube widths ranges pretty wide, run 5000 times for both validation sets



In [35]:
may_opt_rmse = pd.read_csv(r'D:\Ben_wd\THESIS\heatsource\WestFork\ModelRuns\Hayes\calibration\03152024_updatedinputs\hypercube\May\OptRMSE250_May.csv')
#non_zeros_rmse = opt_rmse[opt_rmse['overall rmse'] != 0]
#rmse_sorted = non_zeros_rmse.sort_values(by='0.0', ascending=True)
july_opt_rmse = pd.read_csv(r'D:\Ben_wd\THESIS\heatsource\WestFork\ModelRuns\Hayes\calibration\03152024_updatedinputs\hypercube\July\OptRMSE250_May.csv')
may_opt_rmse
#july_opt_rmse.sort_values(by = 'overall rmse')
july_opt_rmse

,thermal conductivity,thermal diffusivity,hypo thickness,hypo %,sediment porosity,wind function a,wind function b,accretion temp,tree canopy,grass canopy,...,8.5,8.0,7.2,5.7,2.7,2.3,1.2,0.0,overall rmse,nse
0,1.599404,0.006643,0.392113,0.025888,0.708063,3.968939e-09,1.379080e-09,11.674548,0.758095,0.125738,...,0.930131,0.629757,0.601874,0.693377,0.819529,0.803911,0.823677,0.872674,0.777665,0.833760
1,1.515434,0.006768,0.360885,0.037036,0.769855,2.682979e-09,1.637198e-09,12.950378,0.578514,0.275231,...,0.852047,0.797626,0.832077,0.855989,0.987613,0.995990,0.855750,0.907574,0.820834,0.817041
2,1.452116,0.006455,0.380603,0.025712,0.921498,1.692984e-09,1.698956e-09,11.356339,0.655578,0.122538,...,0.907211,0.874972,0.922883,0.941715,1.017600,1.033318,0.871772,0.934795,0.842535,0.805939
3,1.462960,0.006531,0.344347,0.029394,0.809128,3.632656e-09,3.142782e-09,9.818995,0.557218,0.203199,...,1.047811,0.672346,0.649187,0.780380,0.964339,0.913910,0.988738,1.043275,0.864562,0.793018
4,1.540215,0.006285,0.316715,0.024212,0.701666,1.390464e-09,1.623709e-09,10.442421,0.785444,0.299561,...,0.922053,0.879189,0.925850,0.955848,1.036932,1.046785,0.876261,0.945700,0.847592,0.803322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,1.563303,0.006530,0.322926,0.020788,0.910575,1.909421e-09,1.287245e-09,9.918153,0.727546,0.306578,...,0.815156,0.756664,0.798582,0.827414,0.878016,0.877408,0.750611,0.817881,0.756426,0.845198
246,1.480191,0.006677,0.362166,0.025592,0.733543,3.059674e-09,1.310979e-09,10.714725,0.601429,0.171876,...,0.833861,0.716952,0.731548,0.776514,0.811179,0.818676,0.728085,0.793216,0.737069,0.853631
247,1.508031,0.006252,0.385348,0.018325,0.711533,3.708320e-09,1.092726e-09,9.616250,0.719437,0.157209,...,0.933126,0.655698,0.632458,0.720097,0.707330,0.695567,0.742410,0.809495,0.726632,0.855977
248,1.542638,0.006368,0.294953,0.038777,0.645649,2.657051e-09,3.006834e-09,11.402741,0.728842,0.101913,...,0.895196,0.636330,0.633286,0.685059,0.975639,0.929121,0.911983,0.950756,0.826880,0.811127


In [54]:
lhs = pd.read_csv(r'D:\Ben_wd\THESIS\heatsource\WestFork\ModelRuns\Hayes\calibration\03152024_updatedinputs\hypercube\July\hypercube250_update.csv',  index_col=[0])
lhs

,thermal conductivity,thermal diffusivity,hypo thickness,hypo %,sediment porosity,wind function a,wind function b,accretion temp,tree canopy,grass canopy,tree overhang,grass overhang
0,1.599404,0.006643,0.392113,0.025888,0.708063,3.968939e-09,1.379080e-09,11.674548,0.758095,0.125738,0.373314,1.051414
1,1.515434,0.006768,0.360885,0.037036,0.769855,2.682979e-09,1.637198e-09,12.950378,0.578514,0.275231,1.141133,0.193682
2,1.452116,0.006455,0.380603,0.025712,0.921498,1.692984e-09,1.698956e-09,11.356339,0.655578,0.122538,1.772481,0.348392
3,1.462960,0.006531,0.344347,0.029394,0.809128,3.632656e-09,3.142782e-09,9.818995,0.557218,0.203199,0.611550,1.086059
4,1.540215,0.006285,0.316715,0.024212,0.701666,1.390464e-09,1.623709e-09,10.442421,0.785444,0.299561,0.433249,0.446378
...,...,...,...,...,...,...,...,...,...,...,...,...
245,1.563303,0.006530,0.322926,0.020788,0.910575,1.909421e-09,1.287245e-09,9.918153,0.727546,0.306578,0.721770,1.836856
246,1.480191,0.006677,0.362166,0.025592,0.733543,3.059674e-09,1.310979e-09,10.714725,0.601429,0.171876,0.242375,0.646660
247,1.508031,0.006252,0.385348,0.018325,0.711533,3.708320e-09,1.092726e-09,9.616250,0.719437,0.157209,0.135092,1.030669
248,1.542638,0.006368,0.294953,0.038777,0.645649,2.657051e-09,3.006834e-09,11.402741,0.728842,0.101913,1.239119,1.020845


## Calibration/Validation split

In [48]:
#July split + overall rmse

# take random 20% of july sensors for calibration
july_sensors = ['17.5', '16.1', '14.9', '13.8', '12.2', '10.7', '8.5', '8.0', '7.2', '5.7', '2.7', '2.3', '1.2','0.0']
july_select = int(len(july_sensors) * 0.3)
july_random_subset = random.sample(july_sensors, july_select)
july_random_subset

#calculate overall rmse for all of the columns that dont include those left out in the split
july_columns_to_keep = [col for col in july_sensors if col not in july_random_subset]
july_calibration_df = july_opt_rmse[july_columns_to_keep]
july_calibration_df = july_calibration_df.add_suffix('_julyCAL')

#calculate rmse for the validation sensors
july_validation_df = july_opt_rmse[july_random_subset]
july_validation_df = july_validation_df.add_suffix('_julyVAL')
july_validation_df



,2.3_julyVAL,2.7_julyVAL,0.0_julyVAL,10.7_julyVAL
0,0.803911,0.819529,0.872674,0.809448
1,0.995990,0.987613,0.907574,0.849240
2,1.033318,1.017600,0.934795,0.854698
3,0.913910,0.964339,1.043275,0.866749
4,1.046785,1.036932,0.945700,0.836436
...,...,...,...,...
245,0.877408,0.878016,0.817881,0.750120
246,0.818676,0.811179,0.793216,0.744916
247,0.695567,0.707330,0.809495,0.716597
248,0.929121,0.975639,0.950756,0.842236


In [47]:
# May split + overall rmse

# take random 20% of july sensors for calibration
may_sensors = ['17.5', '16.1', '12.2', '10.7', '7.2', '4.4', '1.2', '0.0']
may_select = int(len(may_sensors) * 0.3)
may_random_subset = random.sample(may_sensors, may_select)

#calculate overall rmse for all of the columns that dont include those left out in the split
columns_to_keep = [col for col in may_sensors if col not in may_random_subset]
may_calibration_df = may_opt_rmse[columns_to_keep]
may_calibration_df = may_calibration_df.add_suffix('_mayCAL')

#calculate rmse for the may validation sensors
may_validation_df = may_opt_rmse[may_random_subset]
may_validation_df = may_validation_df.add_suffix('_mayVAL')
may_validation_df

#may_filtered_df['calibration overall rmse'] = may_filtered_df.mean(axis=1)
#may_filtered_df


,0.0_mayVAL,12.2_mayVAL
0,1.556535,1.368456
1,1.178604,1.378577
2,1.088198,1.418838
3,1.997123,1.541038
4,1.077053,1.475906
...,...,...
245,1.059575,1.382999
246,1.189863,1.264846
247,1.404941,1.228078
248,1.734631,1.604411


In [51]:
#Merging june and July validation metrics

validation_merge = pd.merge(may_validation_df, july_validation_df, left_index = True, right_index = True)
validation_merge

,0.0_mayVAL,12.2_mayVAL,2.3_julyVAL,2.7_julyVAL,0.0_julyVAL,10.7_julyVAL
0,1.556535,1.368456,0.803911,0.819529,0.872674,0.809448
1,1.178604,1.378577,0.995990,0.987613,0.907574,0.849240
2,1.088198,1.418838,1.033318,1.017600,0.934795,0.854698
3,1.997123,1.541038,0.913910,0.964339,1.043275,0.866749
4,1.077053,1.475906,1.046785,1.036932,0.945700,0.836436
...,...,...,...,...,...,...
245,1.059575,1.382999,0.877408,0.878016,0.817881,0.750120
246,1.189863,1.264846,0.818676,0.811179,0.793216,0.744916
247,1.404941,1.228078,0.695567,0.707330,0.809495,0.716597
248,1.734631,1.604411,0.929121,0.975639,0.950756,0.842236


In [57]:
#Merging june and July calibration metrics

calibration_merge = pd.merge(may_calibration_df, july_calibration_df, left_index = True, right_index = True)
calibration_merge = pd.merge(lhs, calibration_merge, left_index = True, right_index = True)
calibration_merge

,thermal conductivity,thermal diffusivity,hypo thickness,hypo %,sediment porosity,wind function a,wind function b,accretion temp,tree canopy,grass canopy,...,17.5_julyCAL,16.1_julyCAL,14.9_julyCAL,13.8_julyCAL,12.2_julyCAL,8.5_julyCAL,8.0_julyCAL,7.2_julyCAL,5.7_julyCAL,1.2_julyCAL
0,1.599404,0.006643,0.392113,0.025888,0.708063,3.968939e-09,1.379080e-09,11.674548,0.758095,0.125738,...,0.539072,0.714351,1.068796,0.859211,0.721500,0.930131,0.629757,0.601874,0.693377,0.823677
1,1.515434,0.006768,0.360885,0.037036,0.769855,2.682979e-09,1.637198e-09,12.950378,0.578514,0.275231,...,0.583829,0.691247,0.810424,0.682500,0.789777,0.852047,0.797626,0.832077,0.855989,0.855750
2,1.452116,0.006455,0.380603,0.025712,0.921498,1.692984e-09,1.698956e-09,11.356339,0.655578,0.122538,...,0.606997,0.649118,0.733872,0.653058,0.793488,0.907211,0.874972,0.922883,0.941715,0.871772
3,1.462960,0.006531,0.344347,0.029394,0.809128,3.632656e-09,3.142782e-09,9.818995,0.557218,0.203199,...,0.527721,0.792613,1.144793,0.931389,0.780617,1.047811,0.672346,0.649187,0.780380,0.988738
4,1.540215,0.006285,0.316715,0.024212,0.701666,1.390464e-09,1.623709e-09,10.442421,0.785444,0.299561,...,0.615684,0.665620,0.765086,0.628683,0.766160,0.922053,0.879189,0.925850,0.955848,0.876261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,1.563303,0.006530,0.322926,0.020788,0.910575,1.909421e-09,1.287245e-09,9.918153,0.727546,0.306578,...,0.591101,0.652063,0.781104,0.607662,0.686179,0.815156,0.756664,0.798582,0.827414,0.750611
246,1.480191,0.006677,0.362166,0.025592,0.733543,3.059674e-09,1.310979e-09,10.714725,0.601429,0.171876,...,0.582399,0.651986,0.797978,0.668132,0.663522,0.833861,0.716952,0.731548,0.776514,0.728085
247,1.508031,0.006252,0.385348,0.018325,0.711533,3.708320e-09,1.092726e-09,9.616250,0.719437,0.157209,...,0.554482,0.663135,0.948292,0.777485,0.616675,0.933126,0.655698,0.632458,0.720097,0.742410
248,1.542638,0.006368,0.294953,0.038777,0.645649,2.657051e-09,3.006834e-09,11.402741,0.728842,0.101913,...,0.537895,0.787869,1.133326,0.860088,0.797533,0.895196,0.636330,0.633286,0.685059,0.911983


In [ ]:
#overall calibration rmse
july_calibration_df['calibration overall rmse'] = july_calibration_df.mean(axis=1)

#overall validation rmse
july_validation_df['validation overall rmse'] = july_validation_df.mean(axis=1)
july_validation_d